In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('financial_loan_clean.csv')

In [ ]:
df.shape
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38576 entries, 0 to 38575
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     38576 non-null  int64  
 1   address_state          38576 non-null  object 
 2   application_type       38576 non-null  object 
 3   emp_length             38576 non-null  object 
 4   emp_title              37138 non-null  object 
 5   grade                  38576 non-null  object 
 6   home_ownership         38576 non-null  object 
 7   issue_date             38576 non-null  object 
 8   last_credit_pull_date  38576 non-null  object 
 9   last_payment_date      38576 non-null  object 
 10  loan_status            38576 non-null  object 
 11  next_payment_date      38576 non-null  object 
 12  member_id              38576 non-null  int64  
 13  purpose                38576 non-null  object 
 14  sub_grade              38576 non-null  object 
 15  te

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,...,total_acc,total_payment,term_months,emp_length_years,interest_rate_pct,is_default,end_date,loan_age_months,issue_year,issue_month
0,1077430,GA,INDIVIDUAL,< 1 year,Ryder,C,RENT,11/2/2021,9/13/2021,4/13/2021,...,4,1009,60,0.5,15.27,1,4/13/2021,NaN,2021,11
1,1072053,CA,INDIVIDUAL,9 years,MKC Accounting,E,RENT,1/1/2021,12/14/2021,1/15/2021,...,4,3939,36,9.0,18.64,0,1/15/2021,0.5,2021,1
2,1069243,CA,INDIVIDUAL,4 years,Chemat Technology Inc,C,RENT,5/1/2021,12/12/2021,9/1/2021,...,11,3522,36,4.0,15.96,1,9/1/2021,4.1,2021,5
3,1041756,TX,INDIVIDUAL,< 1 year,barnes distribution,B,MORTGAGE,2/25/2021,12/12/2021,12/3/2021,...,9,4911,60,0.5,10.65,0,12/3/2021,9.4,2021,2
4,1068350,IL,INDIVIDUAL,10+ years,J&J Steel Inc,A,MORTGAGE,1/1/2021,12/14/2021,1/15/2021,...,28,3835,36,10.0,6.03,0,1/15/2021,0.5,2021,1


In [ ]:
date_cols = ['issue_date','last_credit_pull_date'	,'last_payment_date','next_payment_date','end_date']
for col in date_cols:
  df[col] = pd.to_datetime(df[col], dayfirst=True, format='mixed')

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38576 entries, 0 to 38575
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     38576 non-null  int64         
 1   address_state          38576 non-null  object        
 2   application_type       38576 non-null  object        
 3   emp_length             38576 non-null  object        
 4   emp_title              37138 non-null  object        
 5   grade                  38576 non-null  object        
 6   home_ownership         38576 non-null  object        
 7   issue_date             38576 non-null  datetime64[ns]
 8   last_credit_pull_date  38576 non-null  datetime64[ns]
 9   last_payment_date      38576 non-null  datetime64[ns]
 10  loan_status            38576 non-null  object        
 11  next_payment_date      38576 non-null  datetime64[ns]
 12  member_id              38576 non-null  int64         
 13  p

In [ ]:
df["income_band"] = pd.qcut(
    df["annual_income"],
    q=5,
    labels=["Very Low", "Low", "Medium", "High", "Very High"]
)


In [ ]:
df["loan_income_ratio"] = df["loan_amount"] / df["annual_income"]


In [ ]:
df["int_rate"].describe()



,int_rate
count,38576.000000
mean,0.120488
std,0.037164
min,0.054200
25%,0.093200
50%,0.118600
75%,0.145900
max,0.245900


In [ ]:
df["interest_rate"] = df["int_rate"]

if df["interest_rate"].max() <= 1:
    df["interest_rate"] = df["interest_rate"] * 100


In [ ]:
df["interest_band"] = pd.qcut(
    df["interest_rate"],
    q=4,
    labels=["Low", "Medium", "High", "Very High"]
)


In [ ]:
df["interest_band"].value_counts()


,count
interest_band,
Low,9797
Medium,9737
Very High,9589
High,9453


#EDA 1 – Default Rate vs Loan Purpose

In [ ]:
purpose_default = (
    df.groupby("purpose")["is_default"]
    .mean()
    .sort_values(ascending=False)
)
purpose_default


,is_default
purpose,
small business,0.256194
renewable_energy,0.180851
educational,0.158730
house,0.155738
other,0.153504
moving,0.150268
medical,0.149925
Debt consolidation,0.145547
vacation,0.142045


#EDA 2 – Income vs Default


In [ ]:
df.groupby("income_band")["is_default"].mean()


/tmp/ipython-input-1059787761.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("income_band")["is_default"].mean()


,is_default
income_band,
Very Low,0.170411
Low,0.148972
Medium,0.144672
High,0.121998
Very High,0.105023


#EDA 3 – Loan Income Ratio

In [ ]:
df.groupby(pd.qcut(df["loan_income_ratio"], 5))["is_default"].mean()


/tmp/ipython-input-91973370.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(pd.qcut(df["loan_income_ratio"], 5))["is_default"].mean()


,is_default
loan_income_ratio,
"(-0.00021100000000000003, 0.0857]",0.107550
"(0.0857, 0.137]",0.109174
"(0.137, 0.196]",0.124854
"(0.196, 0.278]",0.146694
"(0.278, 0.83]",0.203255


#EDA 4 – Interest Rate


In [ ]:
df.groupby("interest_band")["is_default"].mean()


/tmp/ipython-input-2502708461.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("interest_band")["is_default"].mean()


,is_default
interest_band,
Low,0.056548
Medium,0.114922
High,0.152438
Very High,0.231411


#EDA 5 – Loan Age Months

In [ ]:
df.groupby(pd.qcut(df["loan_age_months"], 5))["is_default"].mean()


/tmp/ipython-input-1403669289.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(pd.qcut(df["loan_age_months"], 5))["is_default"].mean()


,is_default
loan_age_months,
"(-0.001, 0.8]",0.245599
"(0.8, 1.1]",0.238037
"(1.1, 1.3]",0.215651
"(1.3, 2.2]",0.166544
"(2.2, 10.2]",0.034626


#SUMARY
1. Business Context

This analysis aims to identify key factors associated with loan default risk using historical loan data. The objective is to understand borrower and loan characteristics that contribute most significantly to default behavior, providing insights to support credit risk assessment and monitoring.

🔹 2. Key Risk Drivers Identified

Based on exploratory analysis, four primary risk drivers were identified:

• Repayment Burden

Loans with higher loan-to-income ratios exhibit substantially higher default rates, indicating that repayment burden is a critical determinant of credit risk.

• Loan Purpose

Small business loans show the highest default rates among all purposes, suggesting higher income volatility and business uncertainty compared to personal consumption loans.

• Borrower Income

Default rates decrease consistently as borrower income increases, highlighting income stability as a protective factor, although income alone does not fully mitigate risk.

• Loan Maturity

Default risk is heavily concentrated in the early stages of loan life and declines sharply after the first two years, indicating that default events are front-loaded.

🔹 3. Supporting Evidence

Default rates more than double for borrowers whose loan amount exceeds approximately 30% of annual income.

Small business loans exhibit default rates exceeding 25%, significantly higher than consumer-focused loan purposes.

Borrowers in the lowest income band default at rates over 17%, compared to approximately 10% for the highest income group.

Loans older than two years show default rates below 5%, compared to over 20% in the first year.

🔹 4. Implications for Decision-Making

These findings suggest that credit risk assessment should prioritize relative loan burden over absolute income levels. Early-stage loan monitoring is critical, particularly for small business borrowers and high loan-to-income segments. The identified risk drivers can inform borrower screening, pricing strategies, and early warning systems.